In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [2]:
# Read stock symbols from CSV
csv_file = "stock_symbol.csv"
symbols_df = pd.read_csv(csv_file)

# Validate CSV data
if 'Symbol' not in symbols_df.columns:
    raise ValueError("CSV file must contain a 'Symbol' column.")

stock_symbols = symbols_df['Symbol'].tolist()


In [3]:

start_date = "2010-01-01"
end_date = pd.Timestamp.today()
data = yf.download(stock_symbols, start = start_date, end = end_date)
data.reset_index(inplace = True)
data

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  6 of 6 completed


Price        Date       Close                                                  \
Ticker                   AAPL        AMZN        GOOG        MSFT        NFLX   
0      2010-01-04    6.440331    6.695000   15.536651   23.254053    7.640000   
1      2010-01-05    6.451465    6.734500   15.468233   23.261560    7.358571   
2      2010-01-06    6.348845    6.612500   15.078298   23.118807    7.617143   
3      2010-01-07    6.337111    6.500000   14.727283   22.878380    7.485714   
4      2010-01-08    6.379241    6.676000   14.923612   23.036160    7.614286   
...           ...         ...         ...         ...         ...         ...   
3837   2025-04-04  188.380005  171.000000  147.740005  359.839996  855.859985   
3838   2025-04-07  181.460007  175.259995  149.240005  357.859985  867.830017   
3839   2025-04-08  172.419998  170.660004  146.580002  354.559998  870.400024   
3840   2025-04-09  198.850006  191.100006  161.059998  390.489990  945.469971   
3841   2025-04-10  190.419998  181.220001  155.369995  381.350006  921.169983   

Price                     High                          ...        Open  \
Ticker        TSLA        AAPL        AMZN        GOOG  ...        GOOG   
0              NaN    6.455077    6.830500   15.605068  ...   15.541608   
1              NaN    6.487878    6.774000   15.563671  ...   15.547310   
2              NaN    6.477044    6.736500   15.514588  ...   15.514588   
3              NaN    6.379844    6.616000   15.121432  ...   15.106558   
4              NaN    6.379843    6.684000   14.954102  ...   14.675223   
...            ...         ...         ...         ...  ...         ...   
3837    239.429993  199.880005  178.139999  153.089996  ...  149.899994   
3838    233.289993  194.149994  183.410004  154.929993  ...  143.389999   
3839    221.860001  190.339996  185.899994  154.440002  ...  153.574997   
3840    272.200012  200.610001  192.649994  161.869995  ...  146.330002   
3841    252.399994  194.779999  186.869995  160.029999  ...  158.759995   

Price                                          Volume                        \
Ticker        MSFT        NFLX        TSLA       AAPL       AMZN       GOOG   
0        23.006110    7.931429         NaN  493729600  151998000   78541293   
1        23.178914    7.652857         NaN  601904800  177038000  120638494   
2        23.201454    7.361429         NaN  552160000  143576000  159744526   
3        23.013620    7.731429         NaN  477131200  220604000  257533695   
4        22.750650    7.498571         NaN  447610800  196610000  189680313   
...            ...         ...         ...        ...        ...        ...   
3837    364.130005  896.500000  255.380005  125910900  123159400   39832200   
3838    350.880005  827.849976  223.779999  160466300  109327100   47823000   
3839    368.260010  912.440002  245.000000  120859500   87710400   35304400   
3840    353.540009  855.929993  224.690002  184395900  116804300   46479500   
3841    382.059998  931.940002  260.000000  121685000   68082500   35218900   

Price                                    
Ticker      MSFT      NFLX         TSLA  
0       38409100  17239600          NaN  
1       49749600  23753100          NaN  
2       58182400  23290400          NaN  
3       50559700   9955400          NaN  
4       51197400   8180900          NaN  
...          ...       ...          ...  
3837    49209900   6798800  181229400.0  
3838    50425000   6656800  183453800.0  
3839    35868900   5625400  171603500.0  
3840    50199700   7498000  219433400.0  
3841    37944100   5116800  180875500.0  

[3842 rows x 31 columns]

In [4]:
# Dictionary to store results for each stock
predictions_dict = {}

for stock_symbol in stock_symbols:
    print(f"Processing stock: {stock_symbol}")
    data = yf.download(stock_symbol, start=start_date, end=end_date)
    data.reset_index(inplace=True)

    # Determine the cutoff point (80% for training, 20% for testing)
    cutoff = int(len(data) * 0.8)

    # Step 1: Prepare the training and testing data using iloc
    data_train = data.iloc[:cutoff][['Open', 'Close']].copy()
    data_test = data.iloc[cutoff:][['Open', 'Close']].copy()

    # Display the shapes of both datasets
    print(f"Total data points: {len(data)}")
    print(f"Training Data shape: {data_train.shape}")
    print(f"Testing Data shape: {data_test.shape}")


Processing stock: GOOG


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Total data points: 3842
Training Data shape: (3073, 2)
Testing Data shape: (769, 2)
Processing stock: AAPL
Total data points: 3842
Training Data shape: (3073, 2)
Testing Data shape: (769, 2)
Processing stock: MSFT



[*********************100%***********************]  1 of 1 completed


Total data points: 3842
Training Data shape: (3073, 2)
Testing Data shape: (769, 2)
Processing stock: AMZN


...

Total data points: 3842
Training Data shape: (3073, 2)
Testing Data shape: (769, 2)
Processing stock: TSLA


[*********************100%***********************]  1 of 1 completed


Total data points: 3720
Training Data shape: (2976, 2)
Testing Data shape: (744, 2)
Processing stock: NFLX


[*********************100%***********************]  1 of 1 completed

Total data points: 3842
Training Data shape: (3073, 2)
Testing Data shape: (769, 2)


In [5]:
# Step 2: Scale the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_train_scaled = scaler.fit_transform(data_train)

In [6]:
# Step 3: Prepare the input (x) and output (y) for the LSTM model
x = []
y = []

# Create sequences of data
time_steps = 60  # Number of time steps to look back

for i in range(time_steps, data_train_scaled.shape[0]):
    x.append(data_train_scaled[i - time_steps:i])  # Previous 60 time steps
    y.append(data_train_scaled[i])  # Current Open and Close prices

x, y = np.array(x), np.array(y)

In [7]:
# Step 5: Build the LSTM model
model = Sequential()

model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units=60, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=80, activation='relu', return_sequences=True))
model.add(Dropout(0.4))

model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))

# Output layer to predict both Open and Close prices
model.add(Dense(units=2))  # 2 units for Open and Close prices

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
# Step 6: Train the model
model.fit(x, y, epochs=100, batch_size=32)  # Adjust epochs and batch size as needed

Epoch 1/100
95/95 [==============================] - 18s 155ms/step - loss: 0.0225
Epoch 2/100
95/95 [==============================] - 15s 158ms/step - loss: 0.0067
Epoch 3/100
95/95 [==============================] - 14s 151ms/step - loss: 0.0054
Epoch 4/100
95/95 [==============================] - 14s 150ms/step - loss: 0.0054
Epoch 5/100
95/95 [==============================] - 14s 149ms/step - loss: 0.0050
Epoch 6/100
95/95 [==============================] - 14s 148ms/step - loss: 0.0047
Epoch 7/100
95/95 [==============================] - 15s 158ms/step - loss: 0.0047
Epoch 8/100
95/95 [==============================] - 14s 151ms/step - loss: 0.0041
Epoch 9/100
95/95 [==============================] - 14s 149ms/step - loss: 0.0035
Epoch 10/100
95/95 [==============================] - 16s 166ms/step - loss: 0.0038
Epoch 11/100
95/95 [==============================] - 20s 205ms/step - loss: 0.0036
Epoch 12/100
95/95 [==============================] - 17s 174ms/step - loss: 0.0036
E

In [9]:
# Step 7: Prepare the test data (you can do this in a similar way)
data_test_scaled = scaler.transform(data_test)

x_test = []
for i in range(time_steps, data_test_scaled.shape[0]):
    x_test.append(data_test_scaled[i - time_steps:i])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))

In [10]:
# Step 8: Make predictions
predictions = model.predict(x_test)

23/23 [==============================] - 2s 42ms/step


In [11]:
# Step 9: Inverse transform the predictions to get actual prices
predictions_inverse = scaler.inverse_transform(predictions)

In [12]:
# predictions_inverse will contain two columns: [Predicted Open, Predicted Close]
predicted_open_price = predictions_inverse[:, 0]
predicted_close_price = predictions_inverse[:, 1]

In [13]:
# Save the predicted prices in the dictionary
predictions_dict[stock_symbol] = {
    "Predicted_Open_Prices": predicted_open_price,
    "Predicted_Close_Prices": predicted_close_price
}

In [14]:
# Save the trained model for each stock
model.save(f'stock_price_model_GOOG.h5')
model.save(f'stock_price_model_AAPL.h5')
model.save(f'stock_price_model_MSFT.h5')
model.save(f'stock_price_model_AMZN.h5')
model.save(f'stock_price_model_TSLA.h5')
model.save(f'stock_price_model_NFLX.h5')

In [15]:
# Save predictions to a CSV file
for symbol, prediction in predictions_dict.items():
    df = pd.DataFrame({
        'Predicted_Open': prediction['Predicted_Open_Prices'],
        'Predicted_Close': prediction['Predicted_Close_Prices']
    })
    df.to_csv(f'predictions_GOOG.csv', index=False)
    df.to_csv(f'predictions_AAPL.csv', index=False)
    df.to_csv(f'predictions_MSFT.csv', index=False)
    df.to_csv(f'predictions_AMZN.csv', index=False)
    df.to_csv(f'predictions_TSLA.csv', index=False)
    df.to_csv(f'predictions_NFLX.csv', index=False)

print("All models trained and predictions saved.")

All models trained and predictions saved.
